# Assignment

**The objetive of this assignment is to predict top 5 localities in Nagpur where Average Property Rate per sq yard is highest.**

To accomplish this assignment data is collected from various property websites of Nagpur. I have collected data from mainly 3 websites.They are following:
1. http://property.sulekha.com/nagpur-real-estate-price-trend
2. https://www.makaan.com/price-trends/property-rates-for-buy-in-nagpur?page=3
3. https://www.99acres.com/property-rates-and-price-trends-in-nagpur

Though I have tried my best but it is very diffiult to gain data for each localities from all websites. So there are many blank columns in data.


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,Locality,Apartment and Flats_1,Individual Houses_1,Plot_1,Apartment and Flat_2,Individual House_2,Plot_2,Plot_3,Aparrtment and Flats_3
0,Ambazari,4009.0,NaN,NaN,10119.0,10070.0,7883.0,NaN,NaN
1,Amravati Road,3332.0,1250.0,275.0,6216.0,5077.0,249.0,NaN,NaN
2,Amravati,3435.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Anjani,NaN,NaN,7000.0,NaN,NaN,NaN,NaN,NaN
4,Bajaj Nagar,6098.0,NaN,8000.0,6958.0,NaN,18000.0,NaN,NaN


In [4]:
df.dtypes

Locality                   object
Apartment and Flats_1     float64
Individual Houses_1       float64
Plot_1                    float64
Apartment and Flat_2      float64
Individual House_2        float64
Plot_2                    float64
Plot_3                    float64
Aparrtment and Flats_3    float64
dtype: object

In [5]:
df = df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)

**Creating some new variables(Variable Featuring) :**

In [6]:
df['Apartment and Flat avg'] = df[['Apartment and Flats_1','Apartment and Flat_2','Aparrtment and Flats_3']].mean(axis=1)
df['Individual House avg'] = df[['Individual Houses_1','Individual House_2']].mean(axis=1)
df['Plot avg'] = df[['Plot_1','Plot_2','Plot_3']].mean(axis=1)
df['Avg rate per sq ft'] = df.mean(axis=1)

**Data cleaning on featured variables :**

In [7]:
df['Apartment and Flat avg'] = df['Apartment and Flat avg'].fillna(df['Apartment and Flat avg'].mean())
df['Individual House avg'] = df['Individual House avg'].fillna(df['Individual House avg'].mean())
df['Plot avg'] = df['Plot avg'].fillna(df['Plot avg'].mean())
df['Avg rate per sq ft'] = df['Avg rate per sq ft'].fillna(df['Avg rate per sq ft'].mean())

In [8]:
#features that are used in model builiding
features = ['Apartment and Flat avg','Individual House avg','Plot avg']

In [10]:
#defining a function to find the best model
def prediction_function(train, test):
    estimators = [50, 75, 80, 100, 125, 150, 200, 250, 500]
    final = []
    
    for e in estimators:
        rf = RandomForestRegressor(random_state = 1, n_estimators = e, min_samples_split = 8, min_samples_leaf = 4)
        gbr = GradientBoostingRegressor(random_state = 1, n_estimators = e, min_samples_split = 8, 
                                        min_samples_leaf = 4, learning_rate = 0.1)

        rf.fit(train[features], train['Avg rate per sq ft'])
        predictions_rf = rf.predict(test[features])
        predictions_rf = predictions_rf.astype(int)

        gbr.fit(train[features], train['Avg rate per sq ft'])
        predictions_gbr = gbr.predict(test[features])
        predictions_gbr = predictions_gbr.astype(int)

        

        mse_rf = (np.sqrt(mean_squared_error(test['Avg rate per sq ft'], predictions_rf)), 'RF')
        mse_gbr = (np.sqrt(mean_squared_error(test['Avg rate per sq ft'], predictions_gbr)), 'GBR')
        
        
        print(mse_rf , mse_gbr)
        error_min = min(mse_rf,mse_gbr)
#         print(error_min)
        final.append((error_min, e))
    
    print(final)
    min_final = min(final)
    print("Minimum MSE, regressor to use and number of estimators: "+str(min_final))
    return list(min_final)

In [11]:
#spliting data into test and train data
train = df.iloc[ :75, :]
test = df.iloc[75: , :]

In [12]:
min_final = prediction_function(train, test)

(3440.8440311551253, 'RF') (3058.0218719158615, 'GBR')
(3471.8281132732604, 'RF') (3013.4486351117816, 'GBR')
(3460.2520173724565, 'RF') (3009.3706760052546, 'GBR')
(3457.8267450761878, 'RF') (3000.0756505956488, 'GBR')
(3468.4586456030647, 'RF') (2986.7671663199453, 'GBR')
(3465.101869894952, 'RF') (2975.805122934386, 'GBR')
(3464.2415656071, 'RF') (2961.295541153973, 'GBR')
(3471.0136122662266, 'RF') (2955.7380636040757, 'GBR')
(3466.2002415992406, 'RF') (2944.3021124050324, 'GBR')
[((3058.0218719158615, 'GBR'), 50), ((3013.4486351117816, 'GBR'), 75), ((3009.3706760052546, 'GBR'), 80), ((3000.0756505956488, 'GBR'), 100), ((2986.7671663199453, 'GBR'), 125), ((2975.805122934386, 'GBR'), 150), ((2961.295541153973, 'GBR'), 200), ((2955.7380636040757, 'GBR'), 250), ((2944.3021124050324, 'GBR'), 500)]
Minimum MSE, regressor to use and number of estimators: ((2944.3021124050324, 'GBR'), 500)


In [13]:
e = min_final[1]
regressor = min_final[0][1]

In [14]:
#model to be used
if(regressor == 'RF'):
    reg = RandomForestRegressor(random_state = 1, n_estimators = e, min_samples_split = 8, min_samples_leaf = 4)
elif(regressor == 'GBR'):
    reg = GradientBoostingRegressor(random_state = 1, n_estimators = e, min_samples_split = 8, min_samples_leaf = 4, learning_rate = 0.1)
else:
    rdg = Ridge(alpha = 1,random_state = 1)

reg.fit(train[features],train['Avg rate per sq ft'])
predictions = reg.predict(df[features])
predictions = predictions.astype(int)

In [15]:
result = pd.DataFrame({'Locality':df['Locality'], 'Avg rate per sq ft': predictions})

In [16]:
result.to_csv('Result.csv')

In [17]:
result = result.sort_values(by='Avg rate per sq ft',ascending=False)

In [18]:
print("Top 5 localities on the basis of Avg rate per sq feet are:")
print(result.iloc[:5,:])

Top 5 localities on the basis of Avg rate per sq feet are:
    Avg rate per sq ft          Locality
4                 9723       Bajaj Nagar
12                8350  Chatrapati Nagar
19                8189        Dharampeth
0                 8117          Ambazari
87                7914         Somalwada


**The accuracy of model can be improved we have more data to train and test model.**

### Thank You